In [2]:
%pip install --upgrade --quiet langchain langchain-neo4j langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain.schema.runnable import  RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_neo4j import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_ollama import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_neo4j.vectorstores.neo4j_vector import Neo4jVector
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars
#from langchain_ollama import OllamaEmbeddings
import os
#from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  GraphDatabase, Driver

from dotenv import load_dotenv



In [58]:
load_dotenv(override=True)
# print("OPENAI_API_KEY:", os.getenv("OPENAI_API_KEY"))
# print("NEO4J_URI:", os.getenv("NEO4J_URI"))
# print("NEO4J_USERNAME:", os.getenv("NEO4J_USERNAME"))
# print("NEO4J_PASSWORD:", os.getenv("NEO4J_PASSWORD"))

True

In [5]:
graph = Neo4jGraph() 

In [6]:
loader = TextLoader(file_path="dummytext.txt")
docs = loader.load()




In [7]:
print(len(docs))
print(docs[0].page_content)

1
1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition
In the idyllic village of Santa Caterina, amidst the rolling hills and sun-kissed landscapes of Sicily, lies the genesis of the Caruso family, a lineage intertwined with the island's rich culinary tapestry. The Carusos were not mere inhabitants of the land; they were the keepers of a culinary heritage that spanned generations. Each family member contributed their unique flair, crafting a narrative of flavors that reflected their diverse experiences and deep-seated love for food.

Giovanni Caruso and Maria: The Founding Generation

Giovanni Caruso, Amico's great-grandfather, was a man of the earth. His calloused hands spoke of years spent cultivating the fertile soils of Santa Caterina, producing olives and grapes that were the pride of the region. Giovanni was not just a farmer but an alchemist of flavors, blending the fruits of his labor into exquisite oils and wines. His wife, Maria, was the soul of the kitchen. A mas

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
chunks = text_splitter.split_documents(docs)

In [9]:
print(len(chunks))
print(chunks[0].page_content)

73
1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition


In [10]:

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

llm_graph_transformer = LLMGraphTransformer(llm=llm)

print("Model:", llm.model_name)
graph_documents = llm_graph_transformer.convert_to_graph_documents(chunks)



C:\Users\hites\AppData\Local\Temp\ipykernel_32544\2567363932.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)


Model: gpt-4.1-nano


In [12]:
print(len(graph_documents))
graph_documents[0]

73


GraphDocument(nodes=[Node(id='Love and Tradition', type='Concept', properties={}), Node(id="Amico's Family", type='Person', properties={})], relationships=[Relationship(source=Node(id="Amico's Family", type='Person', properties={}), target=Node(id='Love and Tradition', type='Concept', properties={}), type='HAS_LEGACY', properties={})], source=Document(metadata={'source': 'dummytext.txt'}, page_content='1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition'))

In [13]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


In [18]:
vector_index

In [20]:
vector_retriever = vector_index.as_retriever()

In [22]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    print("Fulltext index already exists.")

# Close the driver connection
driver.close()

Fulltext index already exists.


In [23]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [24]:
parser = PydanticOutputParser(pydantic_object=Entities)
format_instructions = parser.get_format_instructions()

# Double‐escape all braces in the JSON template
escaped_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

entity_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Please return valid JSON that matches this format:\n"
            "```\n"                # open a literal code block
            f"{escaped_instructions}\n"
            "```\n\n"               # close the code block
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)



In [25]:
print(entity_prompt)

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are extracting organization and person entities from the text.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Please return valid JSON that matches this format:\n```\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}\nthe object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.\n\nHere is the output schema:\n```\n{{"description": "Identifying information about entities."

In [39]:
def extract_entities(input: str) -> list[str]:
    # 1) Build the correctly escaped/formatted message list
    messages = entity_prompt.format_messages(question=input)

    # 2) Send those messages to the LLM
    response = llm.invoke(messages)

    # 3) Get the raw string (which should now be valid JSON)
    raw_json = response.content

    # 4) Parse into your Pydantic model
    result = parser.parse(raw_json)
    return result.names


In [40]:
entity_names = extract_entities("Who are Nonna Lucia and Giovanni Caruso?")
print(entity_names)

['Nonna Lucia', 'Giovanni Caruso']


In [53]:
# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = extract_entities(question)
    for entity in entities:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [54]:
print(graph_retriever("Who is Nonna Lucia?"))

Nonna Lucia - IS_ROLE_OF -> custodian of the family's recipes
Nonna Lucia - TAUGHT_IN -> her kitchen
Nonna Lucia - HAS_ROLE -> Mentor
Nonna Lucia - HAS_ROLE -> Matriarch
Nonna Lucia - TAUGHT_IN -> kitchen
Nonna Lucia - IS_CUSTODIAN_OF -> family's recipes
Nonna Lucia - INFLUENCED -> kitchen
Nonna Lucia - TOOK_ROLE_SERIOUSLY -> custodian of the family's recipes
Nonna Lucia - WAS_CUSTODIAN_OF -> family's recipes
Nonna Lucia - IS_A -> Mentor
Nonna Lucia - IS_A -> Matriarch
she - INFLUENCED -> Nonna Lucia
Her kitchen - IS_SACRED_SPACE_OF -> Nonna Lucia
Lucia - IS_CULINARY_SAGE -> Nonna Lucia
Lucia - HAS_ROLE -> Nonna
Lucia - USES -> innovation
Lucia - TAUGHT_IN -> her kitchen
Lucia - WAS_CUSTODIAN_OF -> family's recipes
Lucia - IS_CULINARY_SAGE -> Culinary expertise
Lucia - COMBINES -> traditional recipes and innovation
Lucia - CREATED_PRODUCT -> Bella Vita
Lucia - BROUGHT_TO -> Los Angeles
Lucia - LOCATED_IN -> Los Angeles
Lucia - FOCUSES_ON -> sustainable cooking practices
Lucia - EMPHASI

In [55]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [56]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser() 
)

In [57]:

chain.invoke(input="is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

'Yes, Nonna Lucia is a person who taught her grandchildren about Sicilian cooking.'